In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import timeit
from models import *
from equations import *
from solvers import *

In [2]:
data_file_name = "data.npz"
params = ModelParams.load_from_npz(data_file_name)
model = Model(params)
shocks = model.shocks

In [3]:
model.params.tilde_tau.max()

np.float64(1.915345393)

In [4]:
solver = ModelSolver(model)
solver.solve()

Iter 1: w_min=9.947e-01, w_max=1.002e+00, X_min=1.760e+03, X_max=4.828e+09, Δw=5.310e-03, ΔP=2.220e-16
Iter 2: w_min=9.906e-01, w_max=1.003e+00, X_min=1.759e+03, X_max=4.833e+09, Δw=4.097e-03, ΔP=3.585e-03
Iter 3: w_min=9.878e-01, w_max=1.004e+00, X_min=1.758e+03, X_max=4.838e+09, Δw=2.755e-03, ΔP=3.442e-03
Iter 4: w_min=9.862e-01, w_max=1.004e+00, X_min=1.757e+03, X_max=4.842e+09, Δw=1.645e-03, ΔP=2.808e-03
Iter 5: w_min=9.853e-01, w_max=1.004e+00, X_min=1.756e+03, X_max=4.845e+09, Δw=8.649e-04, ΔP=1.966e-03
Iter 6: w_min=9.849e-01, w_max=1.004e+00, X_min=1.756e+03, X_max=4.847e+09, Δw=4.401e-04, ΔP=1.227e-03
Iter 7: w_min=9.848e-01, w_max=1.005e+00, X_min=1.755e+03, X_max=4.849e+09, Δw=2.945e-04, ΔP=7.866e-04
Iter 8: w_min=9.848e-01, w_max=1.005e+00, X_min=1.755e+03, X_max=4.849e+09, Δw=2.733e-04, ΔP=4.984e-04
Iter 9: w_min=9.849e-01, w_max=1.005e+00, X_min=1.755e+03, X_max=4.850e+09, Δw=1.996e-04, ΔP=3.103e-04
Iter 10: w_min=9.850e-01, w_max=1.005e+00, X_min=1.755e+03, X_max=4.850e+

In [ ]:
# Counterfactual Tariff: U.S.
# U.S.: impose uniform 25% tariff on all imported aluminum products 
target_importers = ['USA']  # one or multiple
country_list = list(params.country_list)
sector_list = list(params.sector_list)
target_importers_index = [country_list.index(importer) for importer in target_importers]  

target_sectors = ['Metal Products']  # one or multiple
target_sectors_index = [sector_list.index(sector) for sector in target_sectors]  

counter_tariff = params.tilde_tau.copy()

for importer_index in target_importers_index:
    for exporter_index in range(len(country_list)):
        for sector_index in target_sectors_index:
            # avoid domestic tariff
            if importer_index != exporter_index:
                counter_tariff[importer_index, exporter_index, sector_index] += 0.75

params.tilde_tau = counter_tariff

model_new = Model(params)
solver = ModelSolver(model_new)
solver.solve()


Iter 1: w_min=9.944e-01, w_max=1.002e+00, X_min=1.760e+03, X_max=4.852e+09, Δw=5.628e-03, ΔP=2.220e-16
Iter 2: w_min=9.900e-01, w_max=1.004e+00, X_min=1.759e+03, X_max=4.862e+09, Δw=4.390e-03, ΔP=3.808e-03
Iter 3: w_min=9.870e-01, w_max=1.005e+00, X_min=1.757e+03, X_max=4.870e+09, Δw=3.003e-03, ΔP=3.677e-03
Iter 4: w_min=9.851e-01, w_max=1.006e+00, X_min=1.756e+03, X_max=4.877e+09, Δw=1.837e-03, ΔP=3.040e-03
Iter 5: w_min=9.841e-01, w_max=1.007e+00, X_min=1.755e+03, X_max=4.881e+09, Δw=1.003e-03, ΔP=2.168e-03
Iter 6: w_min=9.837e-01, w_max=1.007e+00, X_min=1.754e+03, X_max=4.885e+09, Δw=5.168e-04, ΔP=1.388e-03
Iter 7: w_min=9.835e-01, w_max=1.008e+00, X_min=1.754e+03, X_max=4.887e+09, Δw=3.930e-04, ΔP=9.245e-04
Iter 8: w_min=9.835e-01, w_max=1.008e+00, X_min=1.754e+03, X_max=4.889e+09, Δw=3.127e-04, ΔP=5.757e-04
Iter 9: w_min=9.835e-01, w_max=1.008e+00, X_min=1.754e+03, X_max=4.890e+09, Δw=2.085e-04, ΔP=3.673e-04
Iter 10: w_min=9.835e-01, w_max=1.008e+00, X_min=1.754e+03, X_max=4.890e+

In [30]:
percent_change = (model_new.sol.pim_prime - model.sol.pim_prime) / model.sol.pim_prime
percent_change[0, 1, 6]

np.float64(0.013806937173065784)

In [23]:
(model_new.sol.w_hat - model.sol.w_hat) / model.sol.w_hat

array([ 0.00110273, -0.00016643, -0.00015593, -0.0001932 , -0.00046083,
       -0.00019513, -0.00021348, -0.00029328, -0.00023174, -0.00107853,
       -0.00011304, -0.00029413, -0.00035046, -0.00029651, -0.00069871,
       -0.00077597, -0.00031171, -0.00020079, -0.00025064, -0.00023172,
       -0.00020635, -0.00020049, -0.00027201, -0.00101192, -0.00022218,
       -0.0002279 , -0.00023938, -0.00067271, -0.00015514, -0.00021735,
       -0.00019681, -0.00023101, -0.00044819, -0.0002868 , -0.00021984,
       -0.00026697, -0.00070431])